## All Techniques Of Hyper Parameter Optimization

1.GridSearchCV

2.RandomizedSearchCV

3.Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)

4.Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)

5.Optuna- Automate Hyperparameter Tuning

6.Genetic Algorithms (TPOT Classifier)


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df[(df['Glucose']==0) | (df['Insulin']==0) | df['SkinThickness']==0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [5]:
df['Glucose'] = np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin'] = np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness'] = np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])

In [6]:
df[(df['Glucose']==0) | (df['Insulin']==0) | df['SkinThickness']==0 ]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [7]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train,y_train)

rf_predict = rf.predict(X_test)

In [10]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

print(classification_report(y_test,rf_predict))
print()
print(confusion_matrix(y_test,rf_predict))
print()
print(accuracy_score(y_test,rf_predict))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80        99
           1       0.63      0.65      0.64        55

    accuracy                           0.74       154
   macro avg       0.72      0.72      0.72       154
weighted avg       0.74      0.74      0.74       154


[[78 21]
 [19 36]]

0.7402597402597403



#### The main parameters used by a Random Forest Classifier are:

criterion = the function used to evaluate the quality of a split.

max_depth = maximum number of levels allowed in each tree.

max_features = maximum number of features considered when splitting a node.

min_samples_leaf = minimum number of samples which can be stored in a tree leaf.

min_samples_split = minimum number of samples necessary in a node to cause node splitting.

n_estimators = number of trees in the ensemble.

In [11]:
### Manual Hyperparameter Tuning

model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=101)
model.fit(X_train,y_train)

predictions=model.predict(X_test)

print(confusion_matrix(y_test,predictions))
print('\n')
print(accuracy_score(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[81 18]
 [21 34]]


0.7467532467532467


              precision    recall  f1-score   support

           0       0.79      0.82      0.81        99
           1       0.65      0.62      0.64        55

    accuracy                           0.75       154
   macro avg       0.72      0.72      0.72       154
weighted avg       0.74      0.75      0.75       154



In [12]:
[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]

In [13]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'criterion':['entropy','gini']}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [14]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [15]:
rf_randomcv.best_params_

{'n_estimators': 1800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 560,
 'criterion': 'entropy'}

In [16]:
rf_randomcv

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [17]:
rf_randomcv.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=560, max_features='log2',
                       n_estimators=1800)

In [18]:
best_random_grid=rf_randomcv.best_estimator_

In [19]:
from sklearn.metrics import accuracy_score

y_pred=best_random_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print('\n')
print(accuracy_score(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[79 20]
 [17 38]]


0.7597402597402597


              precision    recall  f1-score   support

           0       0.82      0.80      0.81        99
           1       0.66      0.69      0.67        55

    accuracy                           0.76       154
   macro avg       0.74      0.74      0.74       154
weighted avg       0.76      0.76      0.76       154



In [20]:
rf_randomcv.best_params_

{'n_estimators': 1800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 560,
 'criterion': 'entropy'}

In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],

    'max_depth': [rf_randomcv.best_params_['max_depth']],
    
    'max_features': [rf_randomcv.best_params_['max_features']],
    
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200,
                     rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100,
                     rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)

{'criterion': ['entropy'], 'max_depth': [560], 'max_features': ['log2'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [0, 1, 2, 3, 4], 'n_estimators': [1600, 1700, 1800, 1900, 2000]}


In [22]:
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  8.5min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [560],
                         'max_features': ['log2'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [0, 1, 2, 3, 4],
                         'n_estimators': [1600, 1700, 1800, 1900, 2000]},
             verbose=2)

In [23]:
grid_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=560, max_features='log2',
                       n_estimators=2000)

In [24]:
best_grid = grid_search.best_estimator_

In [25]:
best_grid

RandomForestClassifier(criterion='entropy', max_depth=560, max_features='log2',
                       n_estimators=2000)

In [26]:
y_pred=best_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print('\n')
print(accuracy_score(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[79 20]
 [17 38]]


0.7597402597402597


              precision    recall  f1-score   support

           0       0.82      0.80      0.81        99
           1       0.66      0.69      0.67        55

    accuracy                           0.76       154
   macro avg       0.74      0.74      0.74       154
weighted avg       0.76      0.76      0.76       154




## Automated Hyperparameter Tuning

Automated Hyperparameter Tuning can be done by using techniques such as

1.Bayesian Optimization

2.Gradient Descent

3.Evolutionary Algorithms

### Bayesian Optimization

Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

Objective Function = defines the loss function to minimize.
Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).

Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [27]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [28]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }

In [29]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x22d7a808f88>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x22d7a80a408>,
 'max_features': <hyperopt.pyll.base.Apply at 0x22d7a80aa88>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x22d7a80aec8>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x22d7a80c388>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x22d7a80cd48>}

In [30]:

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [31]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|███████████████████████████████████████████████| 80/80 [16:19<00:00, 12.24s/trial, best loss: -0.7833933093429295]


{'criterion': 1,
 'max_depth': 390.0,
 'max_features': 2,
 'min_samples_leaf': 0.010473858676785672,
 'min_samples_split': 0.007337695660802443,
 'n_estimators': 6}

In [32]:

crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
log2
1500


In [33]:

best['min_samples_leaf']

0.010473858676785672

In [34]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)

predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print('\n')
print(accuracy_score(y_test,predictionforest))

print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[81 18]
 [16 39]]


0.7792207792207793
              precision    recall  f1-score   support

           0       0.84      0.82      0.83        99
           1       0.68      0.71      0.70        55

    accuracy                           0.78       154
   macro avg       0.76      0.76      0.76       154
weighted avg       0.78      0.78      0.78       154



## Genetic Algorithms

##### Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.


Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [35]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
\
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]

# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [36]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)

C:\Users\User\anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



Generation 1 - Current best internal CV score: 0.7882501485442661

Generation 2 - Current best internal CV score: 0.7882501485442661

Generation 3 - Current best internal CV score: 0.793120278414396

Generation 4 - Current best internal CV score: 0.793120278414396

Generation 5 - Current best internal CV score: 0.793120278414396

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=entropy, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=14, n_estimators=1000)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [37]:
tpot_classifier.get_params

<bound method BaseEstimator.get_params of TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                       

In [38]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.7662337662337663


## Optimize hyperparameters of the model using Optuna
The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [41]:

import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [42]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-11-02 09:18:24,826] A new study created in memory with name: no-name-fb519756-d0fb-44b5-8d91-c54bb2e12940
[I 2020-11-02 09:18:28,784] Trial 0 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_c': 1.2702127706939833e-06}. Best is trial 0 with value: 0.6530926191614858.
[I 2020-11-02 09:18:31,759] Trial 1 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_c': 10001.309986371507}. Best is trial 0 with value: 0.6530926191614858.
[I 2020-11-02 09:18:38,847] Trial 2 finished with value: 0.780121154152718 and parameters: {'classifier': 'RandomForest', 'n_estimators': 900, 'max_depth': 76.15199977393446}. Best is trial 2 with value: 0.780121154152718.
[I 2020-11-02 09:18:44,709] Trial 3 finished with value: 0.7833652160051012 and parameters: {'classifier': 'RandomForest', 'n_estimators': 550, 'max_depth': 88.5010659302041}. Best is trial 3 with value: 0.7833652160051012.
[I 2020-11-02 09:18:47,299] Trial 4 finished with v

[I 2020-11-02 09:20:49,139] Trial 35 finished with value: 0.7768611509644509 and parameters: {'classifier': 'RandomForest', 'n_estimators': 630, 'max_depth': 52.67742157667843}. Best is trial 23 with value: 0.7866172485254265.
[I 2020-11-02 09:20:49,234] Trial 36 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_c': 0.009389111785851188}. Best is trial 23 with value: 0.7866172485254265.
[I 2020-11-02 09:20:53,956] Trial 37 finished with value: 0.7817471704128806 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1000, 'max_depth': 71.14915506378246}. Best is trial 23 with value: 0.7866172485254265.
[I 2020-11-02 09:20:59,747] Trial 38 finished with value: 0.7784871672246134 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1270, 'max_depth': 24.181515880921374}. Best is trial 23 with value: 0.7866172485254265.
[I 2020-11-02 09:20:59,822] Trial 39 finished with value: 0.6530926191614858 and parameters: {'classifier': 'SVC', 'svc_

[I 2020-11-02 09:23:01,477] Trial 70 finished with value: 0.7784791965566714 and parameters: {'classifier': 'RandomForest', 'n_estimators': 420, 'max_depth': 16.127770594423716}. Best is trial 53 with value: 0.7866331898613104.
[I 2020-11-02 09:23:05,679] Trial 71 finished with value: 0.780113183484776 and parameters: {'classifier': 'RandomForest', 'n_estimators': 900, 'max_depth': 14.322196879346933}. Best is trial 53 with value: 0.7866331898613104.
[I 2020-11-02 09:23:09,259] Trial 72 finished with value: 0.7768611509644509 and parameters: {'classifier': 'RandomForest', 'n_estimators': 780, 'max_depth': 17.07266563758843}. Best is trial 53 with value: 0.7866331898613104.
[I 2020-11-02 09:23:11,915] Trial 73 finished with value: 0.7736091184441256 and parameters: {'classifier': 'RandomForest', 'n_estimators': 570, 'max_depth': 41.09623330287675}. Best is trial 53 with value: 0.7866331898613104.
[I 2020-11-02 09:23:16,618] Trial 74 finished with value: 0.7849992029332058 and parameters

Accuracy: 0.7866331898613104
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 810, 'max_depth': 22.167110521789635}


In [43]:
trial

FrozenTrial(number=53, value=0.7866331898613104, datetime_start=datetime.datetime(2020, 11, 2, 9, 21, 48, 300388), datetime_complete=datetime.datetime(2020, 11, 2, 9, 21, 52, 285607), params={'classifier': 'RandomForest', 'n_estimators': 810, 'max_depth': 22.167110521789635}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': LogUniformDistribution(high=100, low=10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=53, state=TrialState.COMPLETE)

In [44]:
study.best_params

{'classifier': 'RandomForest',
 'n_estimators': 810,
 'max_depth': 22.167110521789635}

In [45]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=30, n_estimators=330)

In [46]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[79 20]
 [18 37]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.81      0.80      0.81        99
           1       0.65      0.67      0.66        55

    accuracy                           0.75       154
   macro avg       0.73      0.74      0.73       154
weighted avg       0.76      0.75      0.75       154

